<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
    <h1 align="center">Thesis - Bias MAE</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import mae
import pandas as pd
import numpy as np

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # train the algorithm with the training dataset
    # Now we run the recommender:
    preds = batch.predict(fittable, test)
    # add the algorithm name for analyzability
    return preds

In [2]:
def model_trainer(item,user,damping):
    all_recs = []
    test_data = []
    bias = Bias(items=item, users=user, damping=damping) # define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('Bias', bias, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)

    metric = mae(all_recs['prediction'], all_recs['rating'])
    
    return metric

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    user = params['user']
    item = params['item']
    damping = params['damping']
    metric = model_trainer(item,user,damping)
    print([item,user,damping,metric],',')
    return {'loss': metric, 'status': STATUS_OK }

space={'item': hp.choice('item', ['True', 'False']),
       'user': hp.choice('user', ['True', 'False']),
       'damping': hp.uniform('damping', 0, 1)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

['True', 'True', 0.997290684437442, 0.7182136513290086]                                                                
,                                                                                                                      
['False', 'True', 0.26187481054936623, 0.718658055787915]                                                              
,                                                                                                                      
['False', 'True', 0.04954637004618534, 0.716925781078949]                                                              
,                                                                                                                      
['True', 'True', 0.5665380865142373, 0.7187304483436446]                                                               
,                                                                                                                      
['False', 'True', 0.7144403919969137, 0.

['False', 'True', 0.3186711668622644, 0.7173053637162261]                                                              
,                                                                                                                      
['False', 'True', 0.07746068260725786, 0.717576694387289]                                                              
,                                                                                                                      
['False', 'False', 0.30017765316908634, 0.7199063592251113]                                                            
,                                                                                                                      
['True', 'True', 0.18425798079376193, 0.7171196862634631]                                                              
,                                                                                                                      
['False', 'True', 0.0639669205164029, 0.

['False', 'True', 0.6489866827475694, 0.7173312194135211]                                                              
,                                                                                                                      
['False', 'True', 0.38298897791170317, 0.7174532644526274]                                                             
,                                                                                                                      
['False', 'True', 0.32760910331173865, 0.717501475023418]                                                              
,                                                                                                                      
['False', 'True', 0.1876146111082163, 0.7191611948771944]                                                              
,                                                                                                                      
['False', 'True', 0.5727706849585052, 0.

In [4]:
array = [['True', 'True', 0.997290684437442, 0.7182136513290086]                                                                
,                                                                                                                      
['False', 'True', 0.26187481054936623, 0.718658055787915]                                                              
,                                                                                                                      
['False', 'True', 0.04954637004618534, 0.716925781078949]                                                              
,                                                                                                                      
['True', 'True', 0.5665380865142373, 0.7187304483436446]                                                               
,                                                                                                                      
['False', 'True', 0.7144403919969137, 0.7184212978890392]                                                              
,                                                                                                                      
['False', 'True', 0.9637577838226962, 0.7148948924317992]                                                              
,                                                                                                                      
['True', 'True', 0.6907707169280269, 0.7189099254070146]                                                               
,                                                                                                                      
['False', 'False', 0.40005254314904437, 0.7175902519795908]                                                            
,                                                                                                                      
['False', 'True', 0.13014618343287265, 0.7174682579772556]                                                             
,                                                                                                                      
['False', 'False', 0.32215142252274354, 0.7177221582559545]                                                            
,                                                                                                                      
['True', 'True', 0.6728166623862116, 0.7170366981408608]                                                               
,                                                                                                                      
['True', 'False', 0.430671940698947, 0.7183729521134038]                                                               
,                                                                                                                      
['True', 'False', 0.021522061764911338, 0.7175862825954713]                                                            
,                                                                                                                      
['False', 'False', 0.4673280570428774, 0.7164735821658432]                                                             
,                                                                                                                      
['False', 'True', 0.781121747694277, 0.717107705421722]                                                                
,                                                                                                                      
['False', 'True', 0.49051990094181463, 0.7151933359459963]                                                             
,                                                                                                                      
['True', 'False', 0.7639168220027132, 0.7174664357873499]                                                              
,                                                                                                                      
['False', 'True', 0.13494662843881144, 0.7185774249444183]                                                             
,                                                                                                                      
['True', 'False', 0.5044116647366444, 0.7170631958029239]                                                              
,                                                                                                                      
['True', 'True', 0.23990250317931983, 0.7168742290836833]                                                              
,                                                                                                                      
['False', 'True', 0.9882235003409777, 0.7164888917819309]                                                              
,                                                                                                                      
['False', 'True', 0.8921804460241625, 0.7164829294652177]                                                              
,                                                                                                                      
['False', 'True', 0.5766559649733969, 0.7205918301455524]                                                              
,                                                                                                                      
['False', 'True', 0.8581441394598692, 0.7180139039305663]                                                              
,                                                                                                                      
['False', 'True', 0.5946626523130812, 0.7169646722847212]                                                              
,                                                                                                                      
['False', 'True', 0.3661247269958234, 0.7188699190765524]                                                              
,                                                                                                                      
['False', 'True', 0.8814352517934037, 0.7188096613334355]                                                              
,                                                                                                                      
['False', 'True', 0.6427659247031414, 0.7176428482988545]                                                              
,                                                                                                                      
['False', 'True', 0.9461345172326663, 0.7169269826359822]                                                              
,                                                                                                                      
['False', 'True', 0.5133792325135991, 0.7175140307371892]                                                              
,                                                                                                                      
['False', 'True', 0.7991976545157307, 0.7171930380988567]                                                              
,                                                                                                                      
['False', 'True', 0.27819485519328874, 0.7142078269383462]                                                             
,                                                                                                                      
['False', 'True', 0.22443132780294295, 0.7182888575606774]                                                             
,                                                                                                                      
['False', 'True', 0.1513493075250715, 0.7180720550268311]                                                              
,                                                                                                                      
['False', 'True', 0.3186711668622644, 0.7173053637162261]                                                              
,                                                                                                                      
['False', 'True', 0.07746068260725786, 0.717576694387289]                                                              
,                                                                                                                      
['False', 'False', 0.30017765316908634, 0.7199063592251113]                                                            
,                                                                                                                      
['True', 'True', 0.18425798079376193, 0.7171196862634631]                                                              
,                                                                                                                      
['False', 'True', 0.0639669205164029, 0.719101061993095]                                                               
,                                                                                                                      
['False', 'False', 0.35771596106255366, 0.7170936621242311]                                                            
,                                                                                                                      
['True', 'True', 0.2651244958629143, 0.7173215907673318]                                                               
,                                                                                                                      
['False', 'True', 0.42392793979726406, 0.7166724709142485]                                                             
,                                                                                                                      
['False', 'False', 0.6243425774433635, 0.7163374361479444]                                                             
,                                                                                                                      
['True', 'True', 0.5405972154173813, 0.7176102962759776]                                                               
,                                                                                                                      
['False', 'False', 0.453085080325913, 0.717442191549104]                                                               
,                                                                                                                      
['False', 'True', 0.7143021564526397, 0.7191694937269867]                                                              
,                                                                                                                      
['True', 'True', 0.3767712509965359, 0.7164987455583267]                                                               
,                                                                                                                      
['False', 'False', 0.19939023706361214, 0.7163400037651289]                                                            
,                                                                                                                      
['False', 'True', 0.005840569959311537, 0.7169986770738591]                                                            
,                                                                                                                      
['True', 'True', 0.9806439103645851, 0.7170473821379161]                                                               
,                                                                                                                      
['False', 'False', 0.7492128149246425, 0.7185967720774694]                                                             
,                                                                                                                      
['False', 'True', 0.2785463152586578, 0.7168598756328357]                                                              
,                                                                                                                      
['True', 'True', 0.8093077431052179, 0.715941255444298]                                                                
,                                                                                                                      
['False', 'True', 0.10671241429931178, 0.7177024558559476]                                                             
,                                                                                                                      
['False', 'False', 0.4098606885261199, 0.718049006699818]                                                              
,                                                                                                                      
['False', 'True', 0.6720461053242897, 0.717592941173858]                                                               
,                                                                                                                      
['True', 'True', 0.4677025306172945, 0.7175470599132714]                                                               
,                                                                                                                      
['False', 'True', 0.9328417763018553, 0.7171850079470573]                                                              
,                                                                                                                      
['False', 'True', 0.5416412102075849, 0.7199069264369323]                                                              
,                                                                                                                      
['False', 'False', 0.024331592180869277, 0.7183078890058809]                                                           
,                                                                                                                      
['True', 'True', 0.8471426820556892, 0.7190504482659645]                                                               
,                                                                                                                      
['False', 'True', 0.3452553328053496, 0.7175608124381934]                                                              
,                                                                                                                      
['False', 'True', 0.23562269848215284, 0.717508823174072]                                                              
,                                                                                                                      
['False', 'False', 0.1489150984864748, 0.7166332608285795]                                                             
,                                                                                                                      
['False', 'True', 0.6079125462799411, 0.7169397046810795]                                                              
,                                                                                                                      
['False', 'True', 0.47505108217260744, 0.7173621427428464]                                                             
,                                                                                                                      
['False', 'True', 0.5431082916716207, 0.7187760165678692]                                                              
,                                                                                                                      
['False', 'True', 0.4934361009918139, 0.7179534511721192]                                                              
,                                                                                                                      
['False', 'True', 0.6489866827475694, 0.7173312194135211]                                                              
,                                                                                                                      
['False', 'True', 0.38298897791170317, 0.7174532644526274]                                                             
,                                                                                                                      
['False', 'True', 0.32760910331173865, 0.717501475023418]                                                              
,                                                                                                                      
['False', 'True', 0.1876146111082163, 0.7191611948771944]                                                              
,                                                                                                                      
['False', 'True', 0.5727706849585052, 0.716339099637076]                                                               
,                                                                                                                      
['False', 'True', 0.4449325303461687, 0.7158808164498699]                                                              
,                                                                                                                      
['False', 'True', 0.29321961704894833, 0.7182135051745191]                                                             
,                                                                                                                      
['False', 'True', 0.7244856630875331, 0.7181911355640164]                                                              
,                                                                                                                      
['True', 'True', 0.11239136314076459, 0.7173445111051614]                                                              
,                                                                                                                      
['False', 'True', 0.2169794498623536, 0.7164334078452144]                                                              
,                                                                                                                      
['False', 'True', 0.25075977244720127, 0.7172042619339258]                                                             
,                                                                                                                      
['False', 'False', 0.5135765913734571, 0.7164436722324452]                                                             
,                                                                                                                      
['True', 'True', 0.39819583897973376, 0.7190393421707735]                                                              
,                                                                                                                      
['False', 'True', 0.4245125190043703, 0.7174710511578292]                                                              
,                                                                                                                      
['False', 'True', 0.6901262986197821, 0.7189048263165827]                                                              
,                                                                                                                      
['False', 'False', 0.3117156872453195, 0.7187556464629631]                                                             
,                                                                                                                      
['False', 'True', 0.3436467316979571, 0.7180198854360005]                                                              
,                                                                                                                      
['True', 'True', 0.5935955254655179, 0.7183514018274001]                                                               
,                                                                                                                      
['False', 'True', 0.27267669633779246, 0.7154491783339307]                                                             
,                                                                                                                      
['False', 'True', 0.17173555567084214, 0.7180370400658526]                                                             
,                                                                                                                      
['False', 'False', 0.8201783830775258, 0.7156815206140613]                                                             
,                                                                                                                      
['True', 'True', 0.9149954358323821, 0.7175907913906334]                                                               
,                                                                                                                      
['False', 'True', 0.7494727602388043, 0.7176428826926858]                                                              
,                                                                                                                      
['False', 'True', 0.0768875273736652, 0.7176139046520642]                                                              
,                                                                                                                      
['False', 'False', 0.7776772142423772, 0.7174246618972303]                                                             
,                                                                                                                      
['False', 'True', 0.6435544341722406, 0.718092809147159]                                                               
,                                                                                                                      
['True', 'True', 0.03903596937934506, 0.7187891074182584]                                                              
,                                                                                                                      
['False', 'True', 0.5528915301325982, 0.7166179185330109]                                                              
,                                                                                                                      
['False', 'True', 0.4865933945960391, 0.718657857030564]                                                               
,                                                                                                                      
['False', 'False', 0.5151319294339927, 0.7175760383982194]                                                             
,                                                                                                                      
['False', 'True', 0.9612418198425139, 0.7187821848566355]                                                              
,                                                                                                                      
['True', 'True', 0.39422922853434195, 0.718956958224362] ]

In [5]:
metric = pd.DataFrame(array)
metric.columns = ['items','users','damping','MAE']
metric.head(5)

,items,users,damping,MAE
0,True,True,0.997291,0.718214
1,False,True,0.261875,0.718658
2,False,True,0.049546,0.716926
3,True,True,0.566538,0.718730
4,False,True,0.714440,0.718421


In [6]:
metric.sort_values(by=['MAE'],ascending=True).head(10)

,items,users,damping,MAE
31,False,True,0.278195,0.714208
5,False,True,0.963758,0.714895
15,False,True,0.490520,0.715193
86,False,True,0.272677,0.715449
88,False,False,0.820178,0.715682
73,False,True,0.444933,0.715881
52,True,True,0.809308,0.715941
42,False,False,0.624343,0.716337
72,False,True,0.572771,0.716339
47,False,False,0.199390,0.716340


In [7]:
metric.to_csv('metric_mae.csv')